# Попробуем TF-IDF и Word2Vec

In [1]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score

class ChatbotModel:
    """
    Класс для предсказания тегов и ответов на вопросы с использованием моделей TF-IDF и Word2Vec.

    Атрибуты:
    ----------
    data : dict
        Словарь с данными, включающий вопросы, ответы и соответствующие теги.
    tfidf_vectorizer : TfidfVectorizer
        Объект TfidfVectorizer для преобразования текстов в TF-IDF векторы.
    w2v_model : Word2Vec
        Обученная модель Word2Vec для векторизации текстов.
    label_encoder : LabelEncoder
        Объект LabelEncoder для преобразования текстовых тегов в числовые.
    tfidf_matrix : sparse matrix
        Матрица TF-IDF для вопросов в данных.
    """

    def __init__(self, data):
        """
        Инициализация модели, включает обучение TF-IDF и Word2Vec моделей, а также кодирование тегов.

        Параметры:
        ----------
        data : dict
            Словарь с данными, включающий вопросы, ответы и соответствующие теги.
        """
        self.data = data
        self.tfidf_vectorizer = TfidfVectorizer()
        self.w2v_model = None
        self.label_encoder = LabelEncoder()

        # Векторизация вопросов для TF-IDF
        self.tfidf_matrix = self.tfidf_vectorizer.fit_transform(self.data['вопрос'])

        # Обучение модели Word2Vec
        processed_texts = [simple_preprocess(text) for text in self.data['вопрос']]
        self.w2v_model = Word2Vec(sentences=processed_texts, vector_size=100, window=5, min_count=1, workers=4)

        # Кодирование тегов в числовой формат
        self.label_encoder.fit(self.data['тег'])

    def get_predicted_tag(self, question, method='w2v'):
        """
        Возвращает предсказанный тег, ответ и оценку схожести для заданного вопроса.

        Параметры:
        ----------
        question : str
            Вопрос, на который нужно получить ответ.
        method : str, optional
            Метод, который будет использоваться для предсказания ('tfidf' или 'w2v'). По умолчанию 'w2v'.

        Возвращает:
        ----------
        tuple
            Предсказанный тег, ответ и оценка схожести.
        """
        if method == 'tfidf':
            return self._predict_with_tfidf(question)
        elif method == 'w2v':
            return self._predict_with_w2v(question)
        else:
            raise ValueError("Unknown method specified: use 'tfidf' or 'w2v'")

    def _predict_with_tfidf(self, question):
        """
        Предсказывает тег, ответ и оценку схожести с использованием TF-IDF.

        Параметры:
        ----------
        question : str
            Вопрос, на который нужно получить ответ.

        Возвращает:
        ----------
        tuple
            Предсказанный тег, ответ и оценка схожести.
        """
        question_tfidf = self.tfidf_vectorizer.transform([question])
        similarities = cosine_similarity(question_tfidf, self.tfidf_matrix)
        most_similar_answer_index = similarities.argmax()

        predicted_tag = self.data['тег'][most_similar_answer_index]
        predicted_answer = self.data['ответ'][most_similar_answer_index]
        score = similarities[0][most_similar_answer_index]

        return predicted_tag, predicted_answer, score

    def _predict_with_w2v(self, question):
        """
        Предсказывает тег, ответ и оценку схожести с использованием Word2Vec.

        Параметры:
        ----------
        question : str
            Вопрос, на который нужно получить ответ.

        Возвращает:
        ----------
        tuple
            Предсказанный тег, ответ и оценка схожести.
        """
        processed_question = simple_preprocess(question)
        question_vector = np.mean([self.w2v_model.wv[word] for word in processed_question if word in self.w2v_model.wv], axis=0)

        if question_vector.size == 0 or np.isnan(question_vector).any():
            question_vector = np.zeros(self.w2v_model.vector_size)  # Возвращаем нулевой вектор

        answer_vectors = []
        for answer in self.data['ответ']:
            processed_answer = simple_preprocess(answer)
            answer_vector = np.mean([self.w2v_model.wv[word] for word in processed_answer if word in self.w2v_model.wv], axis=0)
            if answer_vector.size == 0 or np.isnan(answer_vector).any():
                answer_vector = np.zeros(self.w2v_model.vector_size)  # Возвращаем нулевой вектор
            answer_vectors.append(answer_vector)

        similarities = cosine_similarity([question_vector], answer_vectors)
        most_similar_answer_index = similarities.argmax()

        predicted_tag = self.data['тег'][most_similar_answer_index]
        predicted_answer = self.data['ответ'][most_similar_answer_index]
        score = similarities[0][most_similar_answer_index]

        return predicted_tag, predicted_answer, score

    def get_answer(self, question, method='w2v'):
        """
        Возвращает ответ на заданный вопрос с использованием выбранного метода.

        Параметры:
        ----------
        question : str
            Вопрос, на который нужно получить ответ.
        method : str, optional
            Метод, который будет использоваться для предсказания ('tfidf' или 'w2v'). По умолчанию 'w2v'.

        Возвращает:
        ----------
        str
            Предсказанный ответ на заданный вопрос.
        """
        tag, answer, score = self.get_predicted_tag(question, method=method)
        return f"Ответ: {answer} (Тег: {tag}, Оценка: {score:.2f})"

    def evaluate(self, test_questions, real_tags, method='w2v'):
        """
        Оценивает модель по тестовым вопросам и выводит предсказанные теги, ответы и точность.

        Параметры:
        ----------
        test_questions : list of str
            Список тестовых вопросов.
        real_tags : list of str
            Список реальных тегов для тестовых вопросов.
        method : str, optional
            Метод, который будет использоваться для предсказания ('tfidf' или 'w2v'). По умолчанию 'w2v'.

        Возвращает:
        ----------
        tuple
            Точность модели и DataFrame с предсказаниями.
        """
        predicted_tags = []
        predicted_answers = []
        scores = []

        for question in test_questions:
            tag, answer, score = self.get_predicted_tag(question, method=method)
            predicted_tags.append(tag)
            predicted_answers.append(answer)
            scores.append(score)

        # Кодируем реальные и предсказанные теги в числовой формат
        real_tags_encoded = self.label_encoder.transform(real_tags)
        predicted_tags_encoded = self.label_encoder.transform(predicted_tags)

        # Подсчитываем точность
        precision = precision_score(real_tags_encoded, predicted_tags_encoded, average='macro', zero_division=1)

        # Создаем DataFrame для результатов
        results_df = pd.DataFrame({
            'Question': test_questions,
            'Real Tag': real_tags,
            'Predicted Tag ({})'.format(method): predicted_tags,
            'Predicted Answer ({})'.format(method): predicted_answers,
            'Score ({})'.format(method): scores
        })

        return precision, results_df

In [2]:
# Пример использования класса
data = {
    'вопрос': [
        "Как отследить мой заказ?",
        "Какие способы оплаты вы принимаете?",
        "Сколько времени занимает доставка?",
        "Как я могу изменить адрес доставки?",
        "Какие у вас есть акции и скидки?",
        "Как вернуть товар?",
        "Как я могу отменить заказ?",
        "Что делать, если я получил поврежденный товар?",
        "Какой статус моего возврата?",
        "Как зарегистрироваться на вашем сайте?"
    ],
    'ответ': [
        "Вы можете отследить заказ, используя номер отслеживания, который был отправлен вам по электронной почте после отправки заказа.",
        "Мы принимаем к оплате банковские карты, СБП",
        "Доставка занимает от 3 до 7 рабочих дней, в зависимости от вашего местоположения.",
        "Чтобы изменить адрес доставки, пожалуйста, свяжитесь с нашей службой поддержки как можно скорее.",
        "Текущие акции и скидки можно найти на нашей странице 'Специальные предложения'.",
        "Вы можете вернуть товар в течение 30 дней с момента покупки, следуя инструкциям на нашей странице возвратов.",
        "Чтобы отменить заказ, свяжитесь с нашей службой поддержки до того, как заказ будет отправлен.",
        "Если вы получили поврежденный товар, свяжитесь с нашей службой поддержки, и мы поможем вам организовать возврат или замену.",
        "Вы можете проверить статус вашего возврата, войдя в свой аккаунт и перейдя в раздел 'Возвраты'.",
        "Чтобы зарегистрироваться, нажмите на кнопку 'Регистрация' в верхнем правом углу страницы и следуйте инструкциям."
    ],
    'тег': [
        "tracking_order",
        "payment_methods",
        "shipping_info",
        "shipping_info",
        "promotions",
        "returns",
        "cancel_order",
        "returns",
        "returns",
        "account_management"
    ]
}

# Пример вопросов для тестирования
test_questions = [
    "Где мой заказ?",
    "Сколько времени занимает доставка?",
    "Я хочу вернуть товар. Как я это могу сделать?",
    "Я хочу зарегистрировать на вашем сайте? Как я могу это сделать?"
]

# Реальные теги для этих вопросов
real_tags = ["tracking_order", "shipping_info", "returns", "account_management"]


In [3]:
# Создаем экземпляр класса ChatbotModel
chatbot = ChatbotModel(data)

# Оценка для Word2Vec
precision_w2v, df_w2v_results = chatbot.evaluate(test_questions, real_tags, method='w2v')
print("Word2Vec Precision:", precision_w2v)
df_w2v_results


Word2Vec Precision: 0.8


,Question,Real Tag,Predicted Tag (w2v),Predicted Answer (w2v),Score (w2v)
0,Где мой заказ?,tracking_order,cancel_order,"Чтобы отменить заказ, свяжитесь с нашей службо...",0.609194
1,Сколько времени занимает доставка?,shipping_info,shipping_info,"Доставка занимает от 3 до 7 рабочих дней, в за...",0.669798
2,Я хочу вернуть товар. Как я это могу сделать?,returns,returns,Вы можете вернуть товар в течение 30 дней с мо...,0.571576
3,Я хочу зарегистрировать на вашем сайте? Как я ...,account_management,account_management,"Чтобы зарегистрироваться, нажмите на кнопку 'Р...",0.406799


In [4]:
# Оценка для TF-IDF
precision_tfidf, df_tfidf_results = chatbot.evaluate(test_questions, real_tags, method='tfidf')
print("TF-IDF Precision:", precision_tfidf)
df_tfidf_results

TF-IDF Precision: 1.0


,Question,Real Tag,Predicted Tag (tfidf),Predicted Answer (tfidf),Score (tfidf)
0,Где мой заказ?,tracking_order,tracking_order,"Вы можете отследить заказ, используя номер отс...",0.748440
1,Сколько времени занимает доставка?,shipping_info,shipping_info,"Доставка занимает от 3 до 7 рабочих дней, в за...",1.000000
2,Я хочу вернуть товар. Как я это могу сделать?,returns,returns,Вы можете вернуть товар в течение 30 дней с мо...,0.861231
3,Я хочу зарегистрировать на вашем сайте? Как я ...,account_management,account_management,"Чтобы зарегистрироваться, нажмите на кнопку 'Р...",0.796032


Видим, что `w2v` ошибочно определил тег у вопроса `Где мой заказ?`. Попробуем решить эту проблему приведением слов к лемме и стемме, удалению повторяющихся слов.

# Зададим вопросы

In [5]:
# Задаем один вопрос и получаем на него ответ
question = "Как я могу оплатить заказ?"
answer = chatbot.get_answer(question, method='tfidf')
print(answer)

Ответ: Чтобы отменить заказ, свяжитесь с нашей службой поддержки до того, как заказ будет отправлен. (Тег: cancel_order, Оценка: 0.80)


In [6]:
# Задаем один вопрос и получаем на него ответ
question = "Как я могу оплатить заказ?"
answer = chatbot.get_answer(question, method='w2v')
print(answer)

Ответ: Чтобы отменить заказ, свяжитесь с нашей службой поддержки до того, как заказ будет отправлен. (Тег: cancel_order, Оценка: 0.67)


Видим, что здесь оба алогоритма ошиблись. Будет стараться решить эту проблему описаным выше способом.